# Easily generate initial states with Overlapper: PySCF methods

This notebook demonstrates how to use Overlapper to easily execute computational chemistry methods implemented in PySCF, and convert their output to generate initial states for quantum algorithms. 

## **Wavefunction format in Overlapper**

The states will be returned in the sum of Slater determinants (SOS) format, 

$$|\psi\rangle = \sum_{i=1}^D c_i |S_i\rangle,$$

where $c_i$ are normalized coefficients and $|S_i\rangle$'s are Slater determinants with a given configuration of electrons. 

Specifically, the final wavefunction will be a Python dictionary object, with each dictionary entry corresponding to a Slater determinant. The keys of the dictionary `(int_a, int_b)` are tuples of integers whose binary representations (when reversed) give the Fock occupation vector of the alpha and beta electrons of the Slater determinant; and the values of the dictionary are their corresponding normalized coefficients $c_i$. 

For example, in a molecule with 4 electrons and 4 spatial orbitals, one determinant we could write in Fock representation is $|\alpha\alpha\beta\beta\rangle$: the configuration of alpha (spin-up) electrons is $|1100\rangle$ and that of beta (spin-down) is $|0011\rangle$. In our dictionary, this determinant would have the form `{(3,12): 1.}`, because `bin(3) = (00)11` and reversed is `1100`; and `bin(12)=1100` and reversed is `0011`.

While this format may seem cryptic, it is one of the most data-savvy ways of storing the determinant information -- especially when collected in a sparse way through the dictionary object.

## **Use Overlapper to run quantum chemistry calculations in PySCF**

To simplify the process of creating initial states from diverse outputs, Overlapper implements wrappers around a wide variety of quantum chemistry methods. This notebook discusses PySCF-based methods -- for density-matrix renormalization group (DMRG) and semistochastic heat-bath configuration interaction (SHCI) see the following notebooks. 

All these incredibly diverse methods have varying argument requirements and have widely varying outputs -- from matrix-product matrices of DMRG to excitation amplitudes in CCSD, from the truncated space of complete active space configuration interaction (CASCI) to the direct determinant form of SHCI. Overlapper allows the user to call these methods in a unified way, and standardises the return types.

In particular, Overlapper implements a series of methods `do_xxx`, where `xxx` is a name of a method like `cisd` or `dmrg``. All these methods take a PySCF Hartree-Fock object as input plus whatever is needed to setup the solver, and all return four items: the solver object (or something to be converted into a wavefunction), list of energies, list of $\langle S^2 \rangle$ expectation values, and list of $\langle S_z \rangle$ expectation values.

### Configuration interaction / coupled cluster with singles and doubles

Start by creating a `Molecule` in PySCF

In [1]:
from pyscf import gto
mol = gto.M(atom=[["Be", (0,0,0)], ["Be", (1.1,0,0)]], basis='sto3g', spin=0)

#### _Execute wavefunction-based methods_

Next we use Overlapper to execute a Hartree-Fock calculation on the molecule, to obtain molecular orbitals for all post-Hartree-Fock methods below. The `do_hf` method is the only method to take the `Molecule` object: all post-HF methods take the Hartree-Fock solver object as input.

In [2]:
from overlapper.state import do_hf, hf_state
hftype = "rhf"

# run an HF calculation
hf, hf_e, hf_ss, hf_sz = do_hf(mol, hftype)
wf_hf = hf_state(hf)
print(f"HF wavefunction: \n {wf_hf}")
print(f"\nRHF energy: {hf_e[0]:.3f}")
print(f"\nRHF S^2 and Sz: {hf_ss[0]:.3f}, {hf_sz[0]:.3f}")

/home/chiffa/miniconda3/envs/overlapper_v0.0.1/lib/python3.9/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


HF wavefunction: 
 {(15, 15): 1.0}

RHF energy: -28.336

RHF S^2 and Sz: 0.000, 0.000


As expected, for a system with 8 electrons, 10 orbitals and zero spin, the Hartree-Fock state is `bin(15)[::-1] = 1111(000000)`. 

In [3]:
from overlapper.state import do_cisd, cisd_state
# defines tolerance for wavefunction reconstruction
TOL = 1e-1 

# run a CISD calculation
mycisd, mycisd_e, mycisd_ss, mycisd_sz = do_cisd(hf)
wf_cisd = cisd_state(mycisd, tol=TOL)
print(f"CISD wavefunction: \n {wf_cisd}")
print(f"\nCISD energy: {mycisd_e[0]:.3f}")
print(f"\n{hftype[0]}CISD spins: {mycisd_ss[0]:.3f}, {mycisd_sz[0]:.3f}")

CISD wavefunction: 
 {(15, 15): 0.9062595163314698, (27, 27): -0.2916786005034067, (23, 23): 0.20557054828542573, (39, 39): -0.11406923668743202}

CISD energy: -28.399

rCISD spins: 0.000, 0.000


Notice that the energy of the CISD method is below the HF energy, and that its wavefunction now acquires further determinants in addition to the HF one.

In the same way, we can get a CCSD wavefunction

In [4]:
from overlapper.state import do_ccsd, ccsd_state
# defines tolerance for wavefunction reconstruction
TOL = 1e-1 

# run a CCSD calculation
myccsd, myccsd_e, myccsd_ss, myccsd_sz = do_ccsd(hf)
wf_ccsd = ccsd_state(myccsd, tol=TOL)
print(f"CCSD wavefunction: \n {wf_ccsd}")
print(f"\n{hftype[0]}CCSD energy: {myccsd_e[0]:.3f}")
print(f"\n{hftype[0]}CCSD spins: {myccsd_ss[0]:.3f}, {myccsd_sz[0]:.3f}")

CCSD wavefunction: 
 {(15, 15): 0.9151186172633302, (27, 27): -0.28600533396288225, (23, 23): 0.1544551750027599, (39, 39): -0.12370918425574487}

rCCSD energy: -28.401

rCCSD spins: 0.000, 0.000


#### _Compute wavefunction overlap_

We can quantify how different any two wavefunctions are by computing their overlap. In Overlapper, this only takes one line of code.

In [5]:
from overlapper.utils.wf_utils import wf_overlap
ccsd_to_cisd_ovlp = wf_overlap(wf_ccsd, wf_cisd)
print(f"CCSD to CISD overlap: {ccsd_to_cisd_ovlp:.3f}")

CCSD to CISD overlap: 0.959


In this case, CCSD and CISD wavefunctions are very similar to each other, with slight differences coming from the differences in the two ansatze.

### Complete active space configuration interaction (CASCI)

#### _Full space_

The molecule $\text{Be}_2$ we have chosen above is small enough that we can run CASCI in the entire orbitals space, without truncation: in that case it amounts to an exact, full-CI solution

In [6]:
from overlapper.state import do_casci, casci_state
ncas, nelecas = 10, (4,4)
mycasci, mycasci_e, mycasci_ss, mycasci_sz = do_casci(hf, ncas, nelecas)
wf_casci = casci_state(mycasci, tol=TOL)
print(wf_casci)
print(f"\nCASCI energy: {mycasci_e[0]:.3f}")
print(f"\nCASCI spins: {mycasci_ss[0]:.3f}, {mycasci_sz[0]:.3f}")

{(15, 23): 0.685593813487851, (23, 15): 0.6856411620092165}

CASCI energy: -28.396

CASCI spins: 0.000, 0.000


Notice that the solution we get has higher energy than the CISD and CCSD ones we got before! This is because of the particular implementation of CASCI in PySCF: sometimes it is necessary to solve for the few lowest-energy states to get the right ground-state. 

#### _Excited state calculations_

Luckily, this is as easy in Overlapper as passing `nroots = X` to the `do_casci()` method, with `X` the desired number of  states. Excited state calculations are implemented for all methods except CCSD, whose implementation in PySCF does not return a wavefunction for the excited states, but only the energy.

Re-running, we see that we get a state the lowest energy, as expected

In [7]:
from overlapper.state import do_casci, casci_state
ncas, nelecas = 10, (4,4)
mycasci, mycasci_e, mycasci_ss, mycasci_sz = do_casci(hf, ncas, nelecas, nroots=3)
wf_casci = casci_state(mycasci, tol=TOL)
print(wf_casci)
print(f"\nCASCI energy: {mycasci_e[0]:.3f}")
print(f"\nCASCI spins: {mycasci_ss[0]:.3f}, {mycasci_sz[0]:.3f}")

{(15, 15): -0.6340383194485049, (23, 23): -0.6340383194485122, (27, 27): 0.3517676822888245, (39, 39): 0.17026384038385872}

CASCI energy: -28.408

CASCI spins: 0.000, 0.000


#### _Extract wavefunctions for excited states_

We can compare CISD and CCSD to the exact solution by extracting the ground-state wavefunction from our CASCI run.

When we do a calculation with `nroots > 1`, it is ambiguous which state we want to target for wavefunction construction. To differentiate, we pass `state = X` to the `casci_state()` method. This works for all wavefunction constructor methods. By default, `state=0` -- we always target the ground-state. 

Reconstructing the wavefunction and comparing the CISD and CCSD solutions to the exact reference, we can evaluate the quality of the initial state they present.

In [8]:
hf_to_casci_ovlp = wf_overlap(wf_hf, wf_casci)
cisd_to_casci_ovlp = wf_overlap(wf_cisd, wf_casci)
ccsd_to_casci_ovlp = wf_overlap(wf_ccsd, wf_casci)
print(f"HF to CASCI overlap: {hf_to_casci_ovlp:.3f}")
print(f"CISD to CASCI overlap: {cisd_to_casci_ovlp:.3f}")
print(f"CCSD to CASCI overlap: {ccsd_to_casci_ovlp:.3f}")

HF to CASCI overlap: 0.634
CISD to CASCI overlap: 0.827
CCSD to CASCI overlap: 0.800


In this case, it is clear that the CCSD wavefunction is the best initial state out of the three.

Another analysis tool Overlapper provides is a comparison of the subspaces spanned by the determinants of each wavefunction. It can evaluate how many unique determinants appear in one wavefunction that do not appear in the other one.

In [9]:
# check how many determinants from wf 1 are NOT in wf 2, and vice versa
# as a percentage of all determinants present
from overlapper.utils.wf_utils import compare_subspaces
num1, num2 = compare_subspaces(wf_cisd, wf_ccsd, tol=TOL)
print(f"WF 1 is {num1*100:.0f}% unique")
print(f"WF 2 is {num2*100:.0f}% unique")

WF 1 is 0% unique
WF 2 is 0% unique


This result tells us that the wavefunctions are operating in the same subspace of determinants, and the only reason the are not identical is because of the coefficients on the determinants.

We could also check how much overlap there is between the first excited state from CISD and the first excited state we obtain from our full-space CASCI calculation.

In [10]:
wf_casci1ex = casci_state(mycasci, state=1, tol=TOL)
mycisd, mycisd_e, mycisd_ss, mycisd_sz = do_cisd(hf, nroots=3)
wf_cisd1ex = cisd_state(mycisd, state=1, tol=TOL)

cisd1ex_to_casci1ex_ovlp = wf_overlap(wf_cisd1ex, wf_casci1ex)
print(f"CISD to CASCI overlap: {cisd1ex_to_casci1ex_ovlp:.3f}")

CISD to CASCI overlap: 0.000


Wavefunction-based methods that are fairly accurate for the ground state can be quite inaccurate for excited states.

#### _Active space_

With Overlapper we can also freely perform active space calculations, by restricting the number of active electrons and orbitals used with CASCI. The process of obtaining the wavefunction automatically determines the frozen electrons and orbitals and adds them back, ensuring that CASCI solutions in a smaller space remain comparable to wavefunctions from e.g. CISD / CCSD done in a larger space.

In [11]:
ncas, nelecas = 8, (2,2)
mycasci, mycasci_e, mycasci_ss, mycasci_sz = do_casci(hf, ncas, nelecas, nroots=3)
wf_casci = casci_state(mycasci, tol=TOL)
print(wf_casci)
print(f"\nCASCI(4e,8o) energy: {mycasci_e[0]:.3f}")
print(f"\nCASCI(4e,8o) spins: {mycasci_ss[0]:.3f}, {mycasci_sz[0]:.3f}")

{(15, 15): 0.6339564618627093, (23, 23): 0.6339564618626229, (27, 27): -0.3523458144163838, (39, 39): -0.16982843634070027}

CASCI(4e,8o) energy: -28.407

CASCI(4e,8o) spins: 0.000, 0.000


Notice that the determinants in the final wavefunction have the full number of electrons, even though CASCI was carried out in a smaller active space. As expected, the energy and the wavefunction obtained in a smalelr active space is worse relative to the exact solution.

### Multireference perturbation theory (MRPT)

Multireference perturbation theory works on top of a CASCI solution to correct for the frozen orbitals missing from the active space. It works by modifying in-place the wavefunction coefficients stored in a CASCI solver. As most perturbation theory approaches, it only mildly changes the wavefunction.

In [12]:
from overlapper.state import do_mrpt, mrpt_state
ncas, nelecas = 10, (4,4)
mycasci, mycasci_e, mycasci_ss, mycasci_sz = do_casci(hf, ncas, nelecas, nroots=3)
mymrpt, mymrpt_e, mymrpt_ss, mymrpt_sz = do_mrpt(mycasci, nroots=3)
wf_mrpt = mrpt_state(mymrpt, tol=TOL)
print(wf_mrpt)
print(f"\MRPT energy: {mycasci_e[0]:.3f}")
print(f"\nMRPT spins: {mymrpt_ss[0]:.3f}, {mymrpt_sz[0]:.3f}")

{(15, 15): -0.6340664543115094, (23, 23): -0.6340664543087327, (27, 27): 0.3518136434650603, (39, 39): 0.17188866207027292}
\MRPT energy: -28.408

MRPT spins: 0.000, 0.000
